# Multi-Fidelity Deep Gaussian process benchmark

This notebook replicates the benchmark experiments from the paper:

[Deep Gaussian Processes for Multi-fidelity Modeling (Kurt Cutajar, Mark Pullin, Andreas Damianou, Neil Lawrence, Javier González)](https://arxiv.org/abs/1903.07320)

Note that the code for one of the benchmark models in the paper, "Deep Multi-fidelity Gaussian process", is not publically available and so does not appear in this notebook.

In [1]:
from prettytable import PrettyTable
import numpy as np
import scipy.stats
from sklearn.metrics import mean_squared_error, r2_score
import emukit.examples.multi_fidelity_dgp

from emukit.examples.multi_fidelity_dgp.baseline_model_wrappers import LinearAutoRegressiveModel, NonLinearAutoRegressiveModel, HighFidelityGp

from emukit.core import ContinuousParameter, ParameterSpace
from emukit.experimental_design import LatinDesign
from emukit.examples.multi_fidelity_dgp.multi_fidelity_deep_gp import MultiFidelityDeepGP

from emukit.test_functions.multi_fidelity import (multi_fidelity_borehole_function, multi_fidelity_branin_function,
                                                  multi_fidelity_park_function, multi_fidelity_hartmann_3d,
                                                  multi_fidelity_currin_function)

# Parameters for different benchmark functions

In [2]:
from collections import namedtuple

Function = namedtuple('Function', ['name', 'y_scale', 'noise_level', 'do_x_scaling', 'num_data', 'fcn'])

borehole = Function(name='borehole', y_scale=100, noise_level=[0.05, 0.1], do_x_scaling=True, num_data=[60, 5], 
                    fcn=multi_fidelity_borehole_function)
branin = Function(name='branin', y_scale=1, noise_level=[0., 0., 0.], do_x_scaling=False, num_data=[80, 30, 10], 
                    fcn=multi_fidelity_branin_function)
currin = Function(name='currin', y_scale=1, noise_level=[0., 0.], do_x_scaling=False, num_data=[12, 5], 
                    fcn=multi_fidelity_currin_function)
park = Function(name='park', y_scale=1, noise_level=[0., 0.], do_x_scaling=False, num_data=[30, 5], 
                    fcn=multi_fidelity_park_function)
hartmann_3d = Function(name='hartmann', y_scale=1, noise_level=[0., 0., 0.], do_x_scaling=False, num_data=[80, 40, 20], 
                    fcn=multi_fidelity_hartmann_3d)

In [3]:
# Function to repeat test across different random seeds.

def do_benchmark(fcn_tuple):
    metrics = dict()

    # Some random seeds to use
    seeds = [123, 184, 202, 289, 732]

    for i, seed in enumerate(seeds):
        run_name = str(seed) + str(fcn_tuple.num_data)
        metrics[run_name] = test_function(fcn_tuple, seed)
        print('After ' + str(i+1) + ' runs of ' + fcn_tuple.name)
        print_metrics(metrics)

    return metrics

In [4]:
# Print metrics as table 
def print_metrics(metrics):
    model_names = list(list(metrics.values())[0].keys())
    metric_names = ['r2', 'mnll', 'rmse']
    table = PrettyTable(['model'] + metric_names)

    for name in model_names:
        mean = []
        for metric_name in metric_names:
            mean.append(np.mean([metric[name][metric_name] for metric in metrics.values()]))
        table.add_row([name] + mean)

    print(table)

In [5]:
def test_function(fcn, seed):
    np.random.seed(seed)

    x_test, y_test, X, Y = generate_data(fcn, 1000)

    mf_dgp_fix_lf_mean = MultiFidelityDeepGP(X, Y, n_iter=5000)
    mf_dgp_fix_lf_mean.name = 'mf_dgp_fix_lf_mean'

    models = [HighFidelityGp(X, Y), LinearAutoRegressiveModel(X, Y), NonLinearAutoRegressiveModel(X, Y), mf_dgp_fix_lf_mean]
    return benchmark_models(models, x_test, y_test)

In [6]:
def benchmark_models(models, x_test, y_test):
    metrics = dict()
    for model in models:
        model.optimize()
        y_mean, y_var = model.predict(x_test)
        metrics[model.name] = calculate_metrics(y_test, y_mean, y_var)
        print('+ ######################## +')
        print(model.name, 'r2', metrics[model.name]['r2'])
        print('+ ######################## + ')
    return metrics

In [7]:
def generate_data(fcn_tuple, n_test_points):
    """
    Generates train and test data for
    """
    
    do_x_scaling = fcn_tuple.do_x_scaling
    fcn, space = fcn_tuple.fcn()
    
    # Generate training data
    new_space = ParameterSpace(space._parameters[:-1])
    latin = LatinDesign(new_space)
    X = [latin.get_samples(n) for n in fcn_tuple.num_data]
    
    # Scale X if required
    if do_x_scaling:
        scalings = X[0].std(axis=0)
    else:
        scalings = np.ones(X[0].shape[1])
        
    for x in X:
        x /= scalings
    
    Y = []
    for i, x in enumerate(X):
        Y.append(fcn.f[i](x * scalings))
    
    y_scale = fcn_tuple.y_scale
    
    # scale y and add noise if required
    noise_levels = fcn_tuple.noise_level
    if any([n > 0 for n in noise_levels]):
        for y, std_noise in zip(Y, noise_levels):
            y /= y_scale + std_noise * np.random.randn(y.shape[0], 1)
    
    # Generate test data
    x_test = latin.get_samples(n_test_points)
    x_test /= scalings
    y_test = fcn.f[-1](x_test * scalings)
    y_test /= y_scale

    i_highest_fidelity = (len(fcn_tuple.num_data) - 1) * np.ones((x_test.shape[0], 1))
    x_test = np.concatenate([x_test, i_highest_fidelity], axis=1)
    print(X[1].shape)
    return x_test, y_test, X, Y

In [8]:
def calculate_metrics(y_test, y_mean_prediction, y_var_prediction):
    # R2
    r2 = r2_score(y_test, y_mean_prediction)
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_mean_prediction))
    # Test log likelihood
    mnll = -np.sum(scipy.stats.norm.logpdf(y_test, loc=y_mean_prediction, scale=np.sqrt(y_var_prediction)))/len(y_test)
    return {'r2': r2, 'rmse': rmse, 'mnll': mnll}

In [9]:
metrics = []
metrics.append(do_benchmark(borehole))

(5, 8)
Optimization restart 1/10, f = 2.6780050188822546
Optimization restart 2/10, f = 1.0935264476928328
Optimization restart 3/10, f = 6.857006533137321
Optimization restart 4/10, f = 2.6779984957092537
Optimization restart 5/10, f = 6.857006533050038
Optimization restart 6/10, f = 2.6779982188829115
Optimization restart 7/10, f = 2.524868617934755
Optimization restart 8/10, f = 6.8570065331075085
Optimization restart 9/10, f = 6.857006533126267
Optimization restart 10/10, f = 6.857006511305246
+ ######################## +
hf_gp r2 0.853075000266343
+ ######################## + 


 /Users/marpulli/miniconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:165: RuntimeWarning:overflow encountered in true_divide
 /Users/marpulli/miniconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /Users/marpulli/miniconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:144: RuntimeWarning:overflow encountered in square
 /Users/marpulli/miniconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:145: RuntimeWarning:overflow encountered in square
 /Users/marpulli/miniconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:146: RuntimeWarning:invalid value encountered in add


Optimization restart 1/10, f = -156.8285538162658
Optimization restart 2/10, f = -179.38036874969842
Optimization restart 3/10, f = -163.36705147772054
Optimization restart 4/10, f = -182.28237823155325
Optimization restart 5/10, f = -176.24304083993212


 /Users/marpulli/miniconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:244: RuntimeWarning:invalid value encountered in true_divide


Optimization restart 6/10, f = -119.99978816309434
Optimization restart 7/10, f = -175.37409704195088
Optimization restart 8/10, f = -179.61244375281075
Optimization restart 9/10, f = -182.86672215283113
Optimization restart 10/10, f = -161.18706361447067


...

Optimization restart 1/10, f = -191.13771457815267
Optimization restart 2/10, f = -145.64378159530622
Optimization restart 3/10, f = -129.13467425198428
Optimization restart 4/10, f = -185.7379559753191
Optimization restart 5/10, f = -167.21016944745378
Optimization restart 6/10, f = -182.9160787575003
Optimization restart 7/10, f = -186.7798129508015
Optimization restart 8/10, f = -184.86279072240546
Optimization restart 9/10, f = -107.9576409924531
Optimization restart 10/10, f = -163.6479869895175
+ ######################## +
ar1 r2 0.9999315069185383
+ ######################## + 
Optimization restart 1/10, f = -136.62855225569234
Optimization restart 2/10, f = -136.6285539897515
Optimization restart 3/10, f = -136.6285520436615
Optimization restart 4/10, f = -136.62854176665266
Optimization restart 5/10, f = -136.6285532370645
Optimization restart 6/10, f = -136.62854672011755
Optimization restart 7/10, f = -136.62855250724243
Optimization restart 8/10, f = -136.62855333575612
Opti

 /Users/marpulli/miniconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:166: RuntimeWarning:overflow encountered in true_divide
 /Users/marpulli/miniconda3/lib/python3.6/site-packages/GPy/kern/src/stationary.py:138: RuntimeWarning:invalid value encountered in add


Optimization restart 4/10, f = -8.38410054022058
Optimization restart 5/10, f = 6.259340118373446
Optimization restart 6/10, f = 6.259344442227789
Optimization restart 7/10, f = 6.259344433821681
Optimization restart 8/10, f = 6.259341465731598
Optimization restart 9/10, f = 6.259341904715415
Optimization restart 10/10, f = 0.7803571265329232
+ ######################## +
nargp r2 0.9987292088929406
+ ######################## + 


 /Users/marpulli/miniconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning:Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.


+ ######################## +
mf_dgp_fix_lf_mean r2 0.9980699864341062
+ ######################## + 
After 5 runs of borehole
+--------------------+--------------------+---------------------+----------------------+
|       model        |         r2         |         mnll        |         rmse         |
+--------------------+--------------------+---------------------+----------------------+
|       hf_gp        | 0.3553408100018105 |  322.9443044652573  |  0.3135555055193182  |
|        ar1         | 0.9999004504751143 | -3.9483659816140753 | 0.004473445236622577 |
|       nargp        | 0.9988907685828211 | -2.8525534323669226 | 0.01471285112951268  |
| mf_dgp_fix_lf_mean | 0.9986970376056385 | -1.9576738453062468 | 0.016095532828221362 |
+--------------------+--------------------+---------------------+----------------------+


In [10]:
for (metric) in zip(metrics):
    print(fcn_name)
    print_metrics(metric[0])

NameError: name 'fcn_name' is not defined